# Elemente mit mehreren Ereignissen
Ereignisse können auch zeitgleich auftreten und dann in Gruppen von Ereignissen geordnet werden. Wenn beispielsweise die Reihenfolge der Artikel in einem Warenkorb nicht bestimmt ist, einem Kundenkonto aber in zeitlicher Ordnung mehrere Warenkörbe zugeordnet werden können, ergeben sich auch daraus Muster.

In [ ]:
from tui_dsmt.fpm import SequentialDatabase, SequentialItemset, Itemset, NoneItemset
from tui_dsmt.fpm import orders, gsp_nested_join_example, prefixspan_nested_projection_example

## Inhaltsverzeichnis
- [Beispiel](#Beispiel)
- [GSP Erweiterung](#GSP-Erweiterung)
- [PrefixSpan Erweiterung](#PrefixSpan-Erweiterung)

## Beispiel
Nachfolgend ist eine Transaktionsdatenbank aufgelistet. Die Sequenzen enthalten in zeitlicher Reihenfolge sortierte Einkäufe, die einem bestimmten Kunden zugeordnet wurden. Die Elemente der Sequenzen können dabei mehrere Items besitzen.

In [ ]:
orders

Beim näheren Betrachten fällt auf, dass in zwei Fällen auf den Kauf der Kombination Pizzastein und Teigrolle auch Olivenöl gekauft wurde. Nur einmal wurde aber das Trio mit dem Pizzaschneider vor dem Olivenöl verkauft.

| Subsequenz                                                                                     | Support |
| ---------------------------------------------------------------------------------------------- | ------- |
| $<\!(\text{Backofenreiniger}, \text{Pizzastein}), (\text{Olivenöl})\!>                       $ | $2$     |
| $<\!(\text{Backofenreiniger}, \text{Pizzaschneider}, \text{Pizzastein}), (\text{Olivenöl})\!>$ | $1$     |

Nachfolgend wollen wir die beiden zur vorgestellten Algorithmen erweitern, um solche Muster auch aus Sequenzen, deren Elemente aus mehreren Ereignissen bestehen, zu extrahieren.

## GSP Erweiterung
Für den GSP Algorithmus muss insbesondere der Join Schritt überarbeitet werden. Wir unterscheiden dazu zwei Fälle:

#### Basisfall ($k=2$)
Zwei häufige 1-Sequenzen $<\!(i_1)\!>$ und $<\!(i_2)\!>$ werden zu **zwei** 2-Sequenzen $<\!(i_1), (i_2)\!>$ und $<\!(i_1, i_2)\!>$ kombiniert.

#### Allgemeiner Fall ($k>2$)
Wir erzeugen zwei Teilsequenzen:
- $\overline{s_1}$ entspricht $s_1$ ohne das erste Ereignis.
- $\overline{s_2}$ entspricht $s_2$ ohne das letzte Ereignis.

Wenn nun $\overline{s_1} = \overline{s_2}$ gilt, wird die erste Sequenz $s_1$ um das letzte Ereignis aus $s_2$ erweitert, indem
- das letzte Ereignis aus $s_2$ Teil des letzten Elements von $s_1$ wird, **falls** die beiden letzten Ereignisse aus $s_2$ zum gleichen Element gehören.
- das letzte Ereignis aus $s_2$ ein separates Element in der neuen Sequenz, andernfalls.

#### Beispiele
- $s_1 = <\!(1),(2,3),(4)\!>$ und $s_2 = <\!(2,3),(4,5)\!>$ $\leadsto$ $<\!(1),(2,3),(4,5)\!>$
- $s_1 = <\!(1),(2,3),(4)\!>$ und $s_2 = <\!(2,3),(4),(5)\!>$ $\leadsto$ $<\!(1),(2,3),(4),(5)\!>$

Eine angepasste Join Funktion könnte dann beispielsweise wie folgt aussehen:

In [ ]:
def gsp_join(s1, s2, k):
    # Basisfall
    if k == 2:
        if s1 == s2:
            return [s1 + s2]
        else:
            return [s1 + s2, SequentialItemset(s1[0] + s2[0])]

    # Allgemeiner Fall
    else:
        if len(s1[0]) == 1:
            s1_ = s1[1:]
        else:
            s1_ = Itemset(s1[0][1:]) + s1[1:]

        if len(s2[-1]) == 1:
            s2_ = s2[:-1]
        else:
            s2_ = s2[:-1] + Itemset(s2[-1][:-1])

        if s1_ == s2_:
            if len(s2[-1]) == 2:
                return [s1[:-1] + Itemset(s1[-1] + s2[-1][-1:])]
            else:
                return [s1 + s2[-1:]]

    # kein Join möglich
    return []

In [ ]:
for tid1, s1 in gsp_nested_join_example:
    for tid2, s2 in gsp_nested_join_example:
        if s1 < s2:
            print(f'{s1:<19}', f'{s2:<18}', ' -> ', gsp_join(s1, s2, k=5))

Natürlich muss auch das Finden der 1-Itemsets, das Pruning und das Zählen des Supports angepasst werden.

Die Funktion `nested_subsets` erzeugt alle Teilmengen der Länge `k` einer Sequenz.

In [ ]:
print('Transaktion:', orders[0][1])
print()

for subset in orders[0][1].nested_subsets(k=2):
    print(subset)

## PrefixSpan Erweiterung
Auch PrefixSpan lässt sich anpassen. Dazu muss insbesondere die Projektion der Datenbanken nach den folgenden Regeln abgeändert werden:
- Das erste Auftauchen eines Ereignisses innerhalb eines Elements wird gezählt.
- Ist ein Element nach dem Entfernen des Präfix nicht leer, wird das Präfix durch $\_$ ersetzt.

#### Beispiele
- $<\! (A), (A, B, C), (A, C), (D), (C, F) \!> \xrightarrow{A} <\! (A, B, C), (A, C), (D), (C, F) \!>$
- $<\! (E, F), (A, B), (D, F), (C), (B) \!> \xrightarrow{A} <\! (\_, B), (D, F), (C), (B) \!>$
- $<\! (E, F), (B), (D, F), (C), (B) \!> \xrightarrow{A}$ *Transaktion wird entfernt*

Eine angepasste Projektionsfunktion, die als Wert für entfernte Ereignisse `None` einsetzt, könnte wie folgt aussehen:

In [ ]:
def project_transaction(transaction, prefix):
    for item_i, item in enumerate(transaction, start=1):
        for event_i, event in enumerate(item, start=1):
            if event == prefix:
                if event_i == len(item):
                    return transaction[item_i:]
                else:
                    return SequentialItemset(NoneItemset + item[event_i:], *transaction[item_i:])

    return None

In [ ]:
for _, transaction in prefixspan_nested_projection_example:
    print(f'{transaction:<37}', ' -A-> ', project_transaction(transaction, 'A'))

Ein Problem ergibt sich dann, wenn Ereignisse mehrfach vorkommen.

In [ ]:
orders[0][1]

Wird diese Transaktion mit dem Präfix *Backofenreiniger* projiziert, dann wird die eventuell häufig vorkommende Sequenz *<(Backofenreiniger, Pizzastein)>* nur dann gefunden, wenn auch *<(Backofenreiniger), (Backofenreiniger, Pizzastein)>* häufig ist.

In [ ]:
project_transaction(orders[0][1], 'Backofenreiniger')

Erneut muss die Suche nach 1-Sequenzen sowie das Zählen des Supports angepasst werden. Aber auch in mehreren Elementen vorkommende Ereignisse muss besonders gehandhabt werden.